In [ ]:
from sympy import *
from sympy.utilities.iterables import uniq
import polyOrders
import numpy as np


init_printing(use_latex="mathjax")

x1, x2, x3, xi1, xi2, xi3 = symbols("x1 x2 x3 xi1 xi2 xi3")

f_x1, f_x2, f_x3, f_xi1, f_xi2, f_xi3 = symbols(
    "f_x1 f_x2 f_x3 f_xi1 f_xi2 f_xi3", cls=Function
)


xs = Matrix([x1, x2, x3])
xis = Matrix([xi1, xi2, xi3])

f_xs = Matrix([f_x1(xi1, xi2, xi3), f_x2(xi1, xi2, xi3), f_x3(xi1, xi2, xi3)])
f_xis = Matrix([f_xi1(x1, x2, x3), f_xi2(x1, x2, x3), f_xi3(x1, x2, x3)])

dxs_dxis = f_xs.diff(xis.transpose()).reshape(3, 3)
dxis_dxs = f_xis.diff(xs.transpose()).reshape(3, 3)
display(dxs_dxis)
display(dxis_dxs)
display(tensorcontraction(tensorproduct(dxs_dxis, dxis_dxs), (1, 2)))
# dxs_dxis @ dxis_dxs

In [ ]:
(names2d, names3d) = polyOrders.polyOrder2Name(polyOrders.getPolyOrder())
(dols2d, dols3d) = polyOrders.getPolyOrder()

print(names3d)

def GetSymComponentList(sname):
    xiDs = []
    for name in names3d:
        xiDs.append(Symbol(sname + name))
    return xiDs


def GetSymTensors(xiDs, dols3d):
    xiDiff0 = xiDs[0]
    xiDiff1 = MutableDenseNDimArray.zeros(3)
    for i0 in range(3):
        diffRow = np.zeros((3), dtype=np.int32)
        diffRow[i0] += 1
        (found, iS) = polyOrders.searchRow(dols3d, diffRow)
        assert found
        xiDiff1[i0] = xiDs[iS]
    xiDiff2 = MutableDenseNDimArray.zeros(3, 3)
    for i0 in range(3):
        for i1 in range(3):
            diffRow = np.zeros((3), dtype=np.int32)
            diffRow[i0] += 1
            diffRow[i1] += 1
            (found, iS) = polyOrders.searchRow(dols3d, diffRow)
            assert found
            xiDiff2[i0, i1] = xiDs[iS]
    xiDiff3 = MutableDenseNDimArray.zeros(3, 3, 3)
    for i0 in range(3):
        for i1 in range(3):
            for i2 in range(3):
                diffRow = np.zeros((3), dtype=np.int32)
                diffRow[i0] += 1
                diffRow[i1] += 1
                diffRow[i2] += 1
                (found, iS) = polyOrders.searchRow(dols3d, diffRow)
                assert found
                xiDiff3[i0, i1, i2] = xiDs[iS]
    return (xiDiff0, xiDiff1, xiDiff2, xiDiff3)

xiDs = GetSymComponentList("xi")
etaDs = GetSymComponentList("eta")
zetaDs = GetSymComponentList("zeta")
(xiDiff0, xiDiff1, xiDiff2, xiDiff3) = GetSymTensors(xiDs, dols3d)
(etaDiff0, etaDiff1, etaDiff2, etaDiff3) = GetSymTensors(etaDs, dols3d)
(zetaDiff0, zetaDiff1, zetaDiff2, zetaDiff3) = GetSymTensors(zetaDs, dols3d)
xisDiff0 = Array([xiDiff0, etaDiff0, zetaDiff0])
xisDiff1 = Array([xiDiff1, etaDiff1, zetaDiff1])
xisDiff2 = Array([xiDiff2, etaDiff2, zetaDiff2])
xisDiff3 = Array([xiDiff3, etaDiff3, zetaDiff3])

xDs = GetSymComponentList("x")
yDs = GetSymComponentList("y")
zDs = GetSymComponentList("z")
(xDiff0, xDiff1, xDiff2, xDiff3) = GetSymTensors(xDs, dols3d)
(yDiff0, yDiff1, yDiff2, yDiff3) = GetSymTensors(yDs, dols3d)
(zDiff0, zDiff1, zDiff2, zDiff3) = GetSymTensors(zDs, dols3d)
xsDiff0 = Array([xDiff0, yDiff0, zDiff0])
xsDiff1 = Array([xDiff1, yDiff1, zDiff1])
xsDiff2 = Array([xDiff2, yDiff2, zDiff2])
xsDiff3 = Array([xDiff3, yDiff3, zDiff3])
x = xsDiff0[0]
y = xsDiff0[1]
z = xsDiff0[2]
xi = xisDiff0[0]
eta = xisDiff0[1]
zeta = xisDiff0[2]

display(xisDiff3)

In [ ]:
rep_list1 = [
    (xsDiff1.reshape(3**2)[i], Array(eye(3)).reshape(3**2)[i]) for i in range(3**2)
]
rep_list2 = [
    (xisDiff1.reshape(3**2)[i], Array(eye(3)).reshape(3**2)[i]) for i in range(3**2)
]
rep_list_eye = rep_list1 + rep_list2

M0 = Array(eye(3)) + Array([[0, 0, 0], [0, 0, 0], [1, 0, 0]])
xsTest = tensorcontraction(tensorproduct(M0, xisDiff0), (1, 2)) + Array(
    [xi**2 + zeta**2 + eta**2 + xi*eta, xi**2 - eta**2 - eta*zeta, zeta**2 + eta**2]
)
evPoint_subsList = [(xi, 0), (eta, 0), (zeta, 0)]
xsDiff1TestAna = xsTest.diff(xisDiff0)
xsDiff2TestAna = xsTest.diff(xisDiff0, xisDiff0)
xsDiff3TestAna = xsTest.diff(xisDiff0, xisDiff0, xisDiff0)
xsDiff1Test = xsDiff1TestAna.subs(evPoint_subsList)
xsDiff2Test = xsDiff2TestAna.subs(evPoint_subsList)
xsDiff3Test = xsDiff3TestAna.subs(evPoint_subsList)
xsDiff1Test = permutedims(xsDiff1Test, index_order_old="ij", index_order_new="ji")
xsDiff2Test = permutedims(xsDiff2Test, index_order_old="ijk", index_order_new="kij")
xsDiff3Test = permutedims(xsDiff3Test, index_order_old="ijkl", index_order_new="lijk")
xsDiff2Test[:, 1,0]

The tested expressions:

$$
i,j,k,l; m,n, p,q
$$

$$
\delta_{ij} = \frac{\partial}{\partial x_i } \xi_m  \frac{\partial}{\partial \xi_m } x_j
$$

$$
0_{ijk} = 
\frac{\partial^2}{\partial x_i \partial x_k } \xi_m  \frac{\partial}{\partial \xi_m } x_j
+
 \frac{\partial}{\partial x_i } \xi_m\frac{\partial}{\partial x_k } \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
$$

$$
\begin{aligned}
0_{ijkl} = 
\frac{\partial^3}{\partial x_i \partial x_k \partial x_l} \xi_m  \frac{\partial}{\partial \xi_m } x_j
+
\frac{\partial^2}{\partial x_i \partial x_k } \xi_m \frac{\partial}{\partial x_l } \xi_n  \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
+ \\
\frac{\partial}{\partial x_i } \xi_m\frac{\partial^2}{\partial x_k \partial x_l} \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
+
 \frac{\partial^2}{\partial x_i \partial x_l} \xi_m\frac{\partial}{\partial x_k } \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
+ \\
\frac{\partial}{\partial x_i } \xi_m\frac{\partial}{\partial x_k } \xi_n \frac{\partial}{\partial x_l } \xi_p \frac{\partial^3}{\partial \xi_m \partial \xi_n \partial \xi_p} x_j
\end{aligned} 
$$



In [ ]:
T1 = permutedims(xisDiff1, index_order_old="mi", index_order_new="im")
T2 = permutedims(xsDiff1, index_order_old="jm", index_order_new="mj")
expr1 = tensorcontraction(tensorproduct(T1, T2), (1, 2)) - Array(eye(3))

# display(T1, T2)
display(expr1.subs(rep_list_eye))

expr1Test = expr1.subs(
    [
        (
            xsDiff1.reshape(9)[i],
            xsDiff1Test.reshape(9)[i],
        )
        for i in range(9)
    ]
)
solTest = solve(
    [expr1Test.reshape(3**2)[i] for i in range(3**2)],
    [xisDiff1.reshape(9)[i] for i in range(3**2)],
)
xisDiff1Test = xisDiff1.subs(solTest)
xisDiff1Test

`expr2` is:
$$
0_{ijk} = 
\frac{\partial^2}{\partial x_i \partial x_k } \xi_m  \frac{\partial}{\partial \xi_m } x_j
+
 \frac{\partial}{\partial x_i } \xi_m\frac{\partial}{\partial x_k } \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
$$

In [ ]:
T1 = permutedims(xisDiff2, index_order_old="mik", index_order_new="ikm")
T2 = permutedims(xsDiff1, index_order_old="jm", index_order_new="mj")
expr2 = permutedims(
    tensorcontraction(tensorproduct(T1, T2), (2, 3)),
    index_order_old="ikj",
    index_order_new="ijk",
)

T1 = permutedims(xisDiff1, index_order_old="mi", index_order_new="im")
T2 = permutedims(xisDiff1, index_order_old="nk", index_order_new="kn")
T3 = permutedims(xsDiff2, index_order_old="jmn", index_order_new="mnj")
P1 = tensorcontraction(tensorproduct(T2, T3), (1, 3))  # kmj
P2 = tensorcontraction(tensorproduct(T1, P1), (1, 3))


expr2 = expr2 + permutedims(P2, index_order_old="ikj", index_order_new="ijk")

display(expr2[0, 0, 0])

expr2c = expr2.subs(rep_list_eye)
display(expr2c)


expr2Test = expr2.subs(
    [
        (
            xsDiff1.reshape(9)[i],
            xsDiff1Test.reshape(9)[i],
        )
        for i in range(9)
    ]
    + [
        (
            xisDiff1.reshape(9)[i],
            xisDiff1Test.reshape(9)[i],
        )
        for i in range(9)
    ]
    + [
        (
            xsDiff2.reshape(27)[i],
            xsDiff2Test.reshape(27)[i],
        )
        for i in range(27)
    ]
)


solTest = solve(
    [expr2Test.reshape(3**3)[i] for i in range(3**3)],
    list(uniq([xisDiff2.reshape(3**3)[i] for i in range(3**3)])),
)
xisDiff2Test = xisDiff2.subs(solTest)
display(xisDiff2Test)
display(
    (
        expr2 - permutedims(expr2, index_order_old="ijk", index_order_new="kji")
    ).simplify()
)
expr2

`expr3` is
$$
\begin{aligned}
0_{ijkl} = 
\frac{\partial^3}{\partial x_i \partial x_k \partial x_l} \xi_m  \frac{\partial}{\partial \xi_m } x_j
+
\frac{\partial^2}{\partial x_i \partial x_k } \xi_m \frac{\partial}{\partial x_l } \xi_n  \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
+ \\
\frac{\partial}{\partial x_i } \xi_m\frac{\partial^2}{\partial x_k \partial x_l} \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
+
 \frac{\partial^2}{\partial x_i \partial x_l} \xi_m\frac{\partial}{\partial x_k } \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n} x_j
+ \\
\frac{\partial}{\partial x_i } \xi_m\frac{\partial}{\partial x_k } \xi_n \frac{\partial}{\partial x_l } \xi_p \frac{\partial^3}{\partial \xi_m \partial \xi_n \partial \xi_p} x_j
\end{aligned} 
$$

In [ ]:
T1 = permutedims(xisDiff3, index_order_old="mikl", index_order_new="iklm")
T2 = permutedims(xsDiff1, index_order_old="jm", index_order_new="mj")
expr3 = permutedims(
    tensorcontraction(tensorproduct(T1, T2), (3, 4)),
    index_order_old="iklj",
    index_order_new="ijkl",
)

T1 = permutedims(xisDiff2, index_order_old="mil", index_order_new="ilm")  # 012
T2 = permutedims(xisDiff1, index_order_old="nk", index_order_new="kn")  # 34
T3 = permutedims(xsDiff2, index_order_old="jmn", index_order_new="mnj")  # 567
expr3 = expr3 + permutedims(
    tensorcontraction(tensorproduct(T1, T2, T3), (2, 5), (4, 6)),
    index_order_old="ilkj",
    index_order_new="ijkl",
)


T1 = permutedims(xisDiff2, index_order_old="nkl", index_order_new="kln")  # 012
T2 = permutedims(xisDiff1, index_order_old="mi", index_order_new="im")  # 34
T3 = permutedims(xsDiff2, index_order_old="jmn", index_order_new="mnj")  # 567
expr3 = expr3 + permutedims(
    tensorcontraction(tensorproduct(T1, T2, T3), (2, 6), (4, 5)),
    index_order_old="klij",
    index_order_new="ijkl",
)


T1 = permutedims(xisDiff2, index_order_old="mik", index_order_new="ikm")  # 012
T2 = permutedims(xisDiff1, index_order_old="nl", index_order_new="ln")  # 34
T3 = permutedims(xsDiff2, index_order_old="jmn", index_order_new="mnj")  # 567
expr3 = expr3 + permutedims(
    tensorcontraction(tensorproduct(T1, T2, T3), (2, 5), (4, 6)),
    index_order_old="iklj",
    index_order_new="ijkl",
)

expr3Test = expr3.subs(
    [
        (
            xsDiff1.reshape(9)[i],
            xsDiff1Test.reshape(9)[i],
        )
        for i in range(9)
    ]
    + [
        (
            xisDiff1.reshape(9)[i],
            xisDiff1Test.reshape(9)[i],
        )
        for i in range(9)
    ]
    + [
        (
            xsDiff2.reshape(27)[i],
            xsDiff2Test.reshape(27)[i],
        )
        for i in range(27)
    ]
    + [
        (
            xisDiff2.reshape(27)[i],
            xisDiff2Test.reshape(27)[i],
        )
        for i in range(27)
    ]
    + [
        (
            xsDiff3.reshape(81)[i],
            xsDiff3Test.reshape(81)[i],
        )
        for i in range(81)
    ]
)

solTest = solve(
    [expr3Test.reshape(3**4)[i] for i in range(3**4)],
    list(uniq([xisDiff3.reshape(3**4)[i] for i in range(3**4)])),
)
xisDiff3Test = xisDiff3.subs(solTest)
display(xisDiff3Test)
tmp = tensorcontraction(
        expr3 - permutedims(expr3, index_order_old="ijkl", index_order_new="kjil")
    ).simplify()
S = np.array([tmp.reshape(3**4)[i].evalf() for i in range(3**4)]).sum()
print(S)
expr3

$$
\frac{\partial }{\partial x_i}u = \frac{\partial}{\partial x_i} \xi_m \frac{\partial}{\partial \xi_m}u
$$

$$
\frac{\partial^2 }{\partial x_i \partial x_j}u = 
\frac{\partial^2}{\partial x_i \partial x_j} \xi_m \frac{\partial}{\partial \xi_m}u 
+
\frac{\partial}{\partial x_i} \xi_m \frac{\partial}{\partial x_j} \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n}u
$$

$$
\begin{aligned}
\frac{\partial^3 }{\partial x_i \partial x_j  \partial x_k}u = 
\frac{\partial^3}{\partial x_i \partial x_j  \partial x_k} \xi_m \frac{\partial}{\partial \xi_m}u 
+
\frac{\partial^2}{\partial x_i \partial x_j}  \xi_m \frac{\partial}{\partial x_k} \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n}u 
+
\frac{\partial^2}{\partial x_i \partial x_k} \xi_m \frac{\partial}{\partial x_j} \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n}u
\\+
\frac{\partial}{\partial x_i} \xi_m \frac{\partial^2}{\partial x_j \partial x_k} \xi_n \frac{\partial^2}{\partial \xi_m \partial \xi_n}u
+
\frac{\partial}{\partial x_i} \xi_m \frac{\partial}{\partial x_j} \xi_n \frac{\partial}{\partial x_k} \xi_p \frac{\partial^3}{\partial \xi_m \partial \xi_n \partial \xi_p}u
\end{aligned}
$$